# Getting Started With Evalbench

EvalBench is a flexible framework designed to measure the quality of generative AI (GenAI) workflows around database specific tasks. As of now, it provides a comprehensive set of tools, and modules to evaluate models on NL2SQL tasks, including capability of running and scoring DQL, DML, and DDL queries across multiple supported databases. Its modular, plug-and-play architecture allows you to seamlessly integrate custom components while leveraging a robust evaluation pipeline, result storage, scoring strategies, and dashboarding capabilities.

### Quick Start Example w/ sqllite
#### 1. Clone the EvalBench repository from GitHub:

In [ ]:
!git clone git@github.com:GoogleCloudPlatform/evalbench.git

#### 2. Install Dependencies

In [ ]:
!pip install -r evalbench/requirements.txt

#### 3. Setup Evalbench environment

In [ ]:
cd evalbench

#### 4. Connect with GCP

In [ ]:
import os
os.environ['EVAL_GCP_PROJECT_ID'] = '<put-your-project-id-here>'
os.environ['EVAL_GCP_PROJECT_REGION'] = '<gcp-region-here>'

In [52]:
from google.colab import auth
auth.authenticate_user(project_id=os.environ['EVAL_GCP_PROJECT_ID'])

#### 5. Run Evalbench

In [ ]:
!python3 evalbench/evalbench.py  --experiment_config="datasets/bat/db_blog/run_configs/sqlite/run_dql.yaml"

#### 6. Build a Report

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

results_dir = "results/"

# Find the first folder in the results directory
# NOTE: Change this logic if you have a specific job_id you want to find
first_folder = None
for folder in os.listdir(results_dir):
  folder_path = os.path.join(results_dir, folder)
  if not folder.startswith(".") and os.path.isdir(folder_path):
    first_folder = folder_path
    break

if first_folder:
  summary_file = os.path.join(first_folder, "summary.csv")
  if os.path.exists(summary_file):
    df = pd.read_csv(summary_file)
    df['percentage'] = (df['correct_results_count'] / df['total_results_count']) * 100
    df_sorted = df.sort_values(by='percentage', ascending=False)
    plt.figure(figsize=(8, 6))
    bars = plt.bar(df_sorted['metric_name'], df_sorted['percentage'], color='skyblue')
    plt.xlabel('Metric Name')
    plt.ylabel('Correct Results (%)')
    plt.title('Percentage of Correct Results per Metric')
    plt.ylim(0, 110)
    plt.xticks(rotation=45)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}%',
                ha='center', va='bottom', fontsize=9)
    plt.tight_layout()
    plt.show()
  else:
    print(f"summary.csv not found in {os.path.join(first_folder, 'summary.csv')}.")
else:
  print("No results found.")


Now report on the overall evaluations and their scoring

In [ ]:
from google.colab import data_table

evals_file = os.path.join(first_folder, "evals.csv")
scores_file = os.path.join(first_folder, "scores.csv")
if not os.path.exists(scores_file) or not os.path.exists(evals_file):
  print("No results found.")
  exit()

evals_df = pd.read_csv(evals_file)
scores_df = pd.read_csv(scores_file)
scores_pivot = scores_df.pivot_table(
    index=["id", "job_id"],
    columns="comparator",
    values="score",
    aggfunc="first"
).reset_index()
scores_pivot.columns.name = None
scores_pivot = scores_pivot.rename(columns={
    "returned_sql": "score_returned_sql",
    "llmrater": "score_llmrater",
    "set_match": "score_set_match",
    "exact_match": "score_exact_match"
})
merged_df = pd.merge(evals_df, scores_pivot, on=["id", "job_id"], how="left")
merged_df["score_executable"] = merged_df["generated_error"].isna().astype(int) * 100
final_df = merged_df[[
    "id",
    "nl_prompt",
    "generated_sql",
    "golden_sql",
    "generated_result",
    "golden_result",
    "score_returned_sql",
    "score_executable",
    "score_llmrater",
    "score_set_match",
    "score_exact_match"
]].rename(columns={
    "generated_sql": "generated_query",
    "golden_sql": "golden_query"
})
data_table.enable_dataframe_formatter()
final_df